In [3]:
import argparse
import gzip
import json
import logging
import mlflow
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import balanced_accuracy_score
from torch.utils.data import DataLoader
from tqdm import tqdm, trange

from dataset import MeliChallengeDataset
from utils import PadSequences


logging.basicConfig(
    format="%(asctime)s: %(levelname)s - %(message)s",
    level=logging.INFO
)

In [8]:
## Clasifier

class MLPClassifier(nn.Module):
    def __init__(self,
                 pretrained_embeddings_path,
                 token_to_index,
                 n_labels,
                 hidden_layers=[256, 128],
                 dropout=0.3,
                 vector_size=300,
                 freeze_embedings=True):
        super().__init__()
        with gzip.open(token_to_index, "rt") as fh:
            token_to_index = json.load(fh)
        embeddings_matrix = torch.randn(len(token_to_index), vector_size)
        embeddings_matrix[0] = torch.zeros(vector_size)
        with gzip.open(pretrained_embeddings_path, "rt") as fh:
            next(fh)
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in token_to_index:
                    embeddings_matrix[token_to_index[word]] =\
                        torch.FloatTensor([float(n) for n in vector.split()])
        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        self.hidden_layers = [
            nn.Linear(vector_size, hidden_layers[0])
        ]
        for input_size, output_size in zip(hidden_layers[:-1], hidden_layers[1:]):
            self.hidden_layers.append(
                nn.Linear(input_size, output_size)
            )
        self.dropout = dropout
        self.hidden_layers = nn.ModuleList(self.hidden_layers)
        self.output = nn.Linear(hidden_layers[-1], n_labels)
        self.vector_size = vector_size

    def forward(self, x):
        x = self.embeddings(x)
        x = torch.mean(x, dim=1)
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
            if self.dropout:
                x = F.dropout(x, self.dropout)
        x = self.output(x)
        return x

In [10]:
'''
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--train-data",
                        help="Path to the the training dataset",
                        required=True)
    parser.add_argument("--token-to-index",
                        help="Path to the the json file that maps tokens to indices",
                        required=True)
    parser.add_argument("--pretrained-embeddings",
                        help="Path to the pretrained embeddings file.",
                        required=True)
    parser.add_argument("--language",
                        help="Language working with",
                        required=True)
    parser.add_argument("--test-data",
                        help="If given, use the test data to perform evaluation.")
    parser.add_argument("--validation-data",
                        help="If given, use the validation data to perform evaluation.")
    parser.add_argument("--embeddings-size",
                        default=300,
                        help="Size of the vectors.",
                        type=int)
    parser.add_argument("--hidden-layers",
                        help="Sizes of the hidden layers of the MLP (can be one or more values)",
                        nargs="+",
                        default=[256, 128],
                        type=int)
    parser.add_argument("--dropout",
                        help="Dropout to apply to each hidden layer",
                        default=0.3,
                        type=float)
    parser.add_argument("--epochs",
                        help="Number of epochs",
                        default=1,
                        type=int)
'''

'\nif __name__ == "__main__":\n    parser = argparse.ArgumentParser()\n    parser.add_argument("--train-data",\n                        help="Path to the the training dataset",\n                        required=True)\n    parser.add_argument("--token-to-index",\n                        help="Path to the the json file that maps tokens to indices",\n                        required=True)\n    parser.add_argument("--pretrained-embeddings",\n                        help="Path to the pretrained embeddings file.",\n                        required=True)\n    parser.add_argument("--language",\n                        help="Language working with",\n                        required=True)\n    parser.add_argument("--test-data",\n                        help="If given, use the test data to perform evaluation.")\n    parser.add_argument("--validation-data",\n                        help="If given, use the validation data to perform evaluation.")\n    parser.add_argument("--embeddings-size",\n     

In [17]:
train_data = "../data/meli-challenge-2019/spanish.train.jsonl.gz"
validation_data = "../data/meli-challenge-2019/spanish.validation.jsonl.gz"
test_data = ""
language = "spanish"
pretrained_embeddings = "../data/SBW-vectors-300-min5.txt.gz"
hidden_layers = [256, 128]
dropout = 0.3
embeddings_size = 300
epochs = 6
token_to_index = "../data/meli-challenge-2019/spanish_token_to_index.json.gz"

In [18]:

pad_sequences = PadSequences(
    pad_value=0,
    max_length=None,
    min_length=1
)

logging.info("Building training dataset")
train_dataset = MeliChallengeDataset(
    dataset_path=train_data,
    random_buffer_size=2048  # This can be a hypterparameter
)
train_loader = DataLoader(
    train_dataset,
    batch_size=128,  # This can be a hyperparameter
    shuffle=False,
    collate_fn=pad_sequences,
    drop_last=False
)

if validation_data:
    logging.info("Building validation dataset")
    validation_dataset = MeliChallengeDataset(
        dataset_path=validation_data,
        random_buffer_size=1
    )
    validation_loader = DataLoader(
        validation_dataset,
        batch_size=128,
        shuffle=False,
        collate_fn=pad_sequences,
        drop_last=False
    )
else:
    validation_dataset = None
    validation_loader = None

if test_data:
    logging.info("Building test dataset")
    test_dataset = MeliChallengeDataset(
        dataset_path=test_data,
        random_buffer_size=1
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=128,
        shuffle=False,
        collate_fn=pad_sequences,
        drop_last=False
    )
else:
    test_dataset = None
    test_loader = None

mlflow.set_experiment(f"diplodatos.{language}")

with mlflow.start_run():
    logging.info("Starting experiment")
    # Log all relevent hyperparameters
    mlflow.log_params({
        "model_type": "Multilayer Perceptron",
        "embeddings": pretrained_embeddings,
        "hidden_layers": hidden_layers,
        "dropout": dropout,
        "embeddings_size": embeddings_size,
        "epochs": epochs
    })
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    logging.info("Building classifier")
    model = MLPClassifier(
        pretrained_embeddings_path=pretrained_embeddings,
        token_to_index=token_to_index,
        n_labels=train_dataset.n_labels,
        hidden_layers=hidden_layers,
        dropout=dropout,
        vector_size=embeddings_size,
        freeze_embedings=True  # This can be a hyperparameter
    )
    model = model.to(device)
    loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        model.parameters(),
        lr=1e-3,  # This can be a hyperparameter
        weight_decay=1e-5  # This can be a hyperparameter
    )

    logging.info("Training classifier")
    for epoch in trange(epochs):
        model.train()
        running_loss = []
        for idx, batch in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            data = batch["data"].to(device)
            target = batch["target"].to(device)
            output = model(data)
            loss_value = loss(output, target)
            loss_value.backward()
            optimizer.step()
            running_loss.append(loss_value.item())
        mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epoch)

        if validation_dataset:
            logging.info("Evaluating model on validation")
            model.eval()
            running_loss = []
            targets = []
            predictions = []
            with torch.no_grad():
                for batch in tqdm(validation_loader):
                    data = batch["data"].to(device)
                    target = batch["target"].to(device)
                    output = model(data)
                    running_loss.append(
                        loss(output, target).item()
                    )
                    targets.extend(batch["target"].numpy())
                    predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
                mlflow.log_metric("validation_loss", sum(running_loss) / len(running_loss), epoch)
                mlflow.log_metric("validation_bacc", balanced_accuracy_score(targets, predictions), epoch)

    if test_dataset:
        logging.info("Evaluating model on test")
        model.eval()
        running_loss = []
        targets = []
        predictions = []
        with torch.no_grad():
            for batch in tqdm(test_loader):
                data = batch["data"].to(device)
                target = batch["target"].to(device)
                output = model(data)
                running_loss.append(
                    loss(output, target).item()
                )
                targets.extend(batch["target"].numpy())
                predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
            mlflow.log_metric("test_loss", sum(running_loss) / len(running_loss), epoch)
            mlflow.log_metric("test_bacc", balanced_accuracy_score(targets, predictions), epoch)


INFO: 'diplodatos.spanish' does not exist. Creating a new experiment


  9%|▊         | 3330/38245 [00:16<02:48, 207.78it/s]


 18%|█▊        | 6729/38245 [00:32<02:34, 203.76it/s]


 26%|██▋       | 10081/38245 [00:49<02:16, 206.69it/s]


 35%|███▍      | 13341/38245 [01:05<02:22, 175.25it/s]


...

 87%|████████▋ | 33393/38245 [02:51<00:28, 172.29it/s]


 95%|█████████▌| 36513/38245 [03:07<00:08, 194.56it/s]


100%|██████████| 38245/38245 [03:16<00:00, 194.57it/s]

 27%|██▋       | 2535/9562 [00:06<00:18, 380.36it/s]


 86%|████████▌ | 8200/9562 [00:21<00:03, 380.39it/s]


100%|██████████| 6/6 [23:20<00:00, 233.46s/it]
